In [ ]:
# !pip -q install "transformers>=4.40.0" "accelerate>=0.31.0" "scikit-learn>=1.3.0"

In [3]:
import os, re, math, random, json, unicodedata
from dataclasses import dataclass
from typing import List, Tuple
from collections import Counter
from inspect import signature

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup

@dataclass
class CFG:
    # raw data
    train_csv: str = "/kaggle/input/dsc-uit-2025/vihallu-train.csv"
    test_csv:  str = "/kaggle/input/dsc-uit-2025/vihallu-private-test.csv"
    id_col:    str = "id"
    ctx_col:   str = "context"
    prm_col:   str = "prompt"
    rsp_col:   str = "response"
    lab_col:   str = "label"

    # packer/classifier tokenizer ngân sách 1 chuỗi
    tok_name: str = "vinai/phobert-large"
    max_len_single: int = 256

    # CE teacher (NLI 3 lớp: Entail / Neutral / Contradict)
    ce_backbone: str = "xlm-roberta-base"
    ce_max_len: int = 256
    ce_bs: int = 32
    ce_epochs: int = 4
    ce_lr: float = 2e-5
    ce_wd: float = 0.01
    ce_warmup: float = 0.1
    ce_dropout: float = 0.1

    # Packer (dual-beam ưu tiên hỗ trợ + mâu thuẫn)
    span_sizes: Tuple[int,...] = (2,3,4)
    stride_sent: int = 1
    top_span_pool: int = 48
    boost_num: float = 0.10
    boost_neg: float = 0.05
    conflict_target_ratio: float = 0.40

    # Gate ngưỡng an toàn
    auc_intrinsic_min: float = 0.74
    auc_extrinsic_min: float = 0.66
    meta_f1_min: float = 0.74
    rl_median_min: float = 0.20

    # IO
    outdir: str = "/kaggle/working/ce_gate_pipeline_v3"

    # misc
    seed: int = 42
    valid_ratio: float = 0.10
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

os.makedirs(CFG.outdir, exist_ok=True)

def seed_all(s=CFG.seed):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(s)
seed_all()

# label maps
LABEL2ID = {"no":0, "intrinsic":1, "extrinsic":2}
ID2LABEL = {v:k for k,v in LABEL2ID.items()}

In [4]:
# Đọc raw + chuẩn hoá id -> str
train_df = pd.read_csv(CFG.train_csv)
test_df  = pd.read_csv(CFG.test_csv)
for df in (train_df, test_df):
    df[CFG.id_col] = df[CFG.id_col].astype(str)

assert {CFG.ctx_col, CFG.prm_col, CFG.rsp_col, CFG.lab_col}.issubset(train_df.columns)
assert {CFG.id_col, CFG.ctx_col, CFG.prm_col, CFG.rsp_col}.issubset(test_df.columns)

# Tokenizers
tok_pack = AutoTokenizer.from_pretrained(CFG.tok_name, use_fast=True)     # PhoBERT cho packer/classifier
tok_ce   = AutoTokenizer.from_pretrained(CFG.ce_backbone, use_fast=True)  # XLM-R cho CE teacher

S_SINGLE = tok_pack.num_special_tokens_to_add(pair=False)
EFFECTIVE_MAX_SINGLE = CFG.max_len_single - S_SINGLE

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [5]:
SENT_SPLIT = re.compile(r"(?<=[\.\!\?。！？])\s+|\n+|(?<=\u3002|\uff01|\uff1f)\s+", re.UNICODE)
TOKEN_RE   = re.compile(r"\w+|[^\w\s]", flags=re.UNICODE)
NUM_RE     = re.compile(r'\d+[.,]?\d*')
NEG_WORDS  = set(["không","chẳng","chả","chưa","chẳng hề","chẳng phải","no","not","never"])

def strip_diacritics(s: str) -> str:
    nf = unicodedata.normalize("NFKD", str(s))
    return "".join([c for c in nf if not unicodedata.combining(c)])

def norm(s: str) -> str:
    return strip_diacritics(str(s).strip().lower())

def toks(s: str) -> List[str]:
    return [t for t in TOKEN_RE.findall(norm(s)) if t.strip()]

def sent_tokenize_vi(text: str) -> List[str]:
    if not isinstance(text, str): return []
    parts = [s.strip() for s in re.split(SENT_SPLIT, text.strip()) if s and s.strip()]
    return parts

def build_idf(docs_tokens: List[List[str]]):
    N = max(1, len(docs_tokens)); df = Counter(); total = 0
    for dt in docs_tokens:
        total += len(dt); df.update(set(dt))
    avgdl = total / N
    idf = {t: math.log((N - df_t + 0.5)/(df_t + 0.5) + 1) for t, df_t in df.items()}
    return idf, max(1.0, avgdl)

def bm25_score(q_tokens, d_tokens, idf, avgdl, k1=1.2, b=0.75):
    dq, dd = Counter(q_tokens), Counter(d_tokens)
    dl = len(d_tokens); s = 0.0
    for t in dq.keys():
        f = dd.get(t, 0)
        if f == 0: continue
        idf_t = idf.get(t, 0.0)
        denom = f + k1*(1 - b + b*dl/max(1.0, avgdl))
        s += idf_t * (f*(k1+1)/denom)
    return s

def char_trigrams(s: str):
    s = norm(s)
    return [s[i:i+3] for i in range(max(0, len(s)-2))] if s else []

def soft_char_overlap(A: List[str], B: List[str], tau=0.6) -> float:
    s = 0.0
    for q in A:
        best=0.0; Aq=set(char_trigrams(q))
        for u in B:
            Bu=set(char_trigrams(u))
            if not Aq and not Bu: continue
            sim = len(Aq & Bu) / max(1, len(Aq | Bu))
            best = max(best, sim)
        if best >= tau: s += best
    return s

def rouge_l_lcs(a_tokens, b_tokens):
    A,B=a_tokens,b_tokens
    m,n=len(A),len(B)
    dp=[[0]*(n+1) for _ in range(m+1)]
    for i in range(m):
        Ai=A[i]
        for j in range(n):
            dp[i+1][j+1]= dp[i][j]+1 if Ai==B[j] else max(dp[i][j+1], dp[i+1][j])
    lcs=dp[m][n]
    prec=lcs/max(1,n); rec=lcs/max(1,m)
    return 2*prec*rec/(prec+rec+1e-9)

def numbers(s: str): return set(NUM_RE.findall(norm(s)))
def has_neg(s: str) -> bool: return len(set(toks(s)) & NEG_WORDS) > 0
def join_pr(prompt, response): return f"{str(prompt).strip()} [SEP] {str(response).strip()}"

In [6]:
def make_span_pool(context, prompt, response):
    sents = sent_tokenize_vi(context)
    if not sents: return []
    R, P = toks(response), toks(prompt)
    W = [toks(s) for s in sents]
    idf, avgdl = build_idf(W)
    spans = []
    for k in CFG.span_sizes:
        for st in range(0, len(sents), CFG.stride_sent):
            en = min(len(sents), st+k)
            if en <= st: continue
            txt = " ".join(sents[st:en]); ET = toks(txt)
            s_lex = 0.5*(0.7*soft_char_overlap(R, ET) + 0.3*bm25_score(R, ET, idf, avgdl)) + \
                    0.5*(0.7*soft_char_overlap(P, ET) + 0.3*bm25_score(P, ET, idf, avgdl))
            spans.append((st,en,txt,s_lex))
    spans.sort(key=lambda x: x[3], reverse=True)
    return spans[:CFG.top_span_pool]

def weak_label_for_span(label, response, span_text, s_lex, th_high=0.22, th_low=0.10):
    r_nums = numbers(response); s_nums = numbers(span_text)
    neg_diff = (has_neg(response) != has_neg(span_text))
    num_mis  = (len(r_nums)>0) and (len(s_nums)>0) and (r_nums != s_nums)
    rl = rouge_l_lcs(toks(response), toks(span_text))

    if label == "no":
        if rl >= th_high or s_lex >= th_high or (len(r_nums)>0 and r_nums.issubset(s_nums)):
            return 0  # Entail
        if num_mis or neg_diff:
            return 2  # Contradict
        return 1
    elif label == "intrinsic":
        if num_mis or neg_diff:
            return 2
        if rl >= (th_high + 0.05):
            return 1  # tránh lệch Entail cho intrinsic
        return 1
    else:  # extrinsic
        if rl < th_low and not (len(r_nums)>0 and r_nums.issubset(s_nums)):
            return 1
        if num_mis or neg_diff:
            return 2
        if rl >= th_high:
            return 0
        return 1

pairs = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc="[build CE pairs]"):
    spans = make_span_pool(row[CFG.ctx_col], row[CFG.prm_col], row[CFG.rsp_col])
    PR = join_pr(row[CFG.prm_col], row[CFG.rsp_col])
    for st,en,txt,sx in spans:
        y = weak_label_for_span(row[CFG.lab_col], row[CFG.rsp_col], txt, sx)
        pairs.append((PR, txt, y))

pairs_df = pd.DataFrame(pairs, columns=["A","B","y"])
pairs_df.to_csv(os.path.join(CFG.outdir, "ce_pairs_raw.csv"), index=False)

minc = pairs_df["y"].value_counts().min()
bal_df = pd.concat([pairs_df[pairs_df["y"]==c].sample(minc, random_state=CFG.seed) for c in [0,1,2]],
                   ignore_index=True).sample(frac=1.0, random_state=CFG.seed)
bal_df.to_csv(os.path.join(CFG.outdir, "ce_pairs_balanced.csv"), index=False)

print("Pairs:", len(pairs_df), "| dist:", pairs_df["y"].value_counts().to_dict())
print("Balanced:", len(bal_df), "| dist:", bal_df["y"].value_counts().to_dict())

[build CE pairs]:   0%|          | 0/7000 [00:00<?, ?it/s]

Pairs: 117567 | dist: {1: 43822, 0: 42475, 2: 31270}
Balanced: 93810 | dist: {0: 31270, 1: 31270, 2: 31270}


In [7]:
class CEModel(nn.Module):
    def __init__(self, name, num_labels=3, dropout=0.1):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(name)
        H = self.backbone.config.hidden_size
        self.dropout = nn.Dropout(dropout)
        self.cls = nn.Linear(H, num_labels)

    def forward(self, enc):
        allowed = set(signature(self.backbone.forward).parameters)
        enc = {k: v for k, v in enc.items() if k in allowed}
        out = self.backbone(**enc, return_dict=True).last_hidden_state[:,0,:]
        out = self.dropout(out)
        return self.cls(out)

class PairDS(Dataset):
    def __init__(self, df): self.df=df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        enc = tok_ce(r["A"], r["B"], truncation=True, max_length=CFG.ce_max_len,
                     padding="max_length", return_tensors="pt")
        item = {k:v.squeeze(0) for k,v in enc.items()}
        item["labels"] = torch.tensor(int(r["y"]), dtype=torch.long)
        return item

def make_loader(df, bs, shuffle):
    return DataLoader(PairDS(df), batch_size=bs, shuffle=shuffle, num_workers=0, pin_memory=True)

mask = np.random.rand(len(bal_df)) < 0.9
tr_ce, va_ce = bal_df[mask].reset_index(drop=True), bal_df[~mask].reset_index(drop=True)
tr_loader = make_loader(tr_ce, CFG.ce_bs, True)
va_loader = make_loader(va_ce, CFG.ce_bs*2, False)

ce = CEModel(CFG.ce_backbone, 3, CFG.ce_dropout).to(CFG.device)
opt = torch.optim.AdamW(ce.parameters(), lr=CFG.ce_lr, weight_decay=CFG.ce_wd)
steps = len(tr_loader)*CFG.ce_epochs
sch = get_linear_schedule_with_warmup(opt, int(steps*CFG.ce_warmup), steps)
scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())

best_f1, ce_ckpt = -1.0, os.path.join(CFG.outdir, "teacher.pt")
for ep in range(1, CFG.ce_epochs+1):
    ce.train(); losses=[]
    pbar = tqdm(tr_loader, desc=f"[CE train ep{ep}]")
    for batch in pbar:
        batch = {k:v.to(CFG.device) for k,v in batch.items()}
        opt.zero_grad(set_to_none=True)
        with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
            logits = ce(batch)
            loss = F.cross_entropy(logits, batch["labels"])
        scaler.scale(loss).backward()
        scaler.step(opt); scaler.update(); sch.step()
        losses.append(loss.item()); pbar.set_postfix(loss=np.mean(losses))

    # eval
    ce.eval(); preds=[]; trues=[]
    with torch.no_grad():
        for batch in va_loader:
            lab = batch["labels"].to(CFG.device)
            enc = {k:v.to(CFG.device) for k,v in batch.items() if k!="labels"}
            logits = ce(enc)
            pred = logits.argmax(-1)
            preds += pred.detach().cpu().tolist()
            trues += lab.detach().cpu().tolist()
    f1 = f1_score(trues, preds, average="macro")
    acc= accuracy_score(trues, preds)
    print(f"[CE VAL] Macro-F1={f1:.4f} Acc={acc:.4f}")
    if f1 > best_f1:
        best_f1 = f1
        torch.save(ce.state_dict(), ce_ckpt)
        print("  ↳ saved", ce_ckpt)
print("CE best F1:", best_f1)

2025-10-05 07:38:46.303432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759649926.655949      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759649926.748264      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

[CE train ep1]:   0%|          | 0/2643 [00:00<?, ?it/s]

[CE VAL] Macro-F1=0.9183 Acc=0.9185
  ↳ saved /kaggle/working/ce_gate_pipeline_v3/teacher.pt


[CE train ep2]:   0%|          | 0/2643 [00:00<?, ?it/s]

[CE VAL] Macro-F1=0.9559 Acc=0.9561
  ↳ saved /kaggle/working/ce_gate_pipeline_v3/teacher.pt


[CE train ep3]:   0%|          | 0/2643 [00:00<?, ?it/s]

[CE VAL] Macro-F1=0.9692 Acc=0.9693
  ↳ saved /kaggle/working/ce_gate_pipeline_v3/teacher.pt


[CE train ep4]:   0%|          | 0/2643 [00:00<?, ?it/s]

[CE VAL] Macro-F1=0.9740 Acc=0.9741
  ↳ saved /kaggle/working/ce_gate_pipeline_v3/teacher.pt
CE best F1: 0.9740049000280916


In [8]:
class TempScaler(nn.Module):
    def __init__(self): super().__init__(); self.t = nn.Parameter(torch.ones(1))
    def forward(self, logits): return logits / self.t.clamp(0.5, 5.0)

ce.load_state_dict(torch.load(ce_ckpt, map_location=CFG.device))
ce.eval()
all_logits, all_y = [], []
with torch.no_grad():
    for batch in va_loader:
        y = batch["labels"].to(CFG.device)
        enc = {k:v.to(CFG.device) for k,v in batch.items() if k!="labels"}
        lg = ce(enc)
        all_logits.append(lg); all_y.append(y)
all_logits = torch.cat(all_logits, 0)
all_y = torch.cat(all_y, 0)

temp = TempScaler().to(CFG.device)
optT = torch.optim.LBFGS(temp.parameters(), lr=0.1, max_iter=50)
def _closure():
    optT.zero_grad()
    loss = F.cross_entropy(temp(all_logits), all_y)
    loss.backward()
    return loss
optT.step(_closure)
T_value = float(temp.t.detach().cpu().clamp(0.5,5.0).item())
json.dump({"t": T_value}, open(os.path.join(CFG.outdir, "temp.json"), "w"))
print("Fitted CE temperature:", T_value)

Fitted CE temperature: 1.234139323234558


In [9]:
def load_temperature(path):
    try:
        return float(json.load(open(path,"r"))["t"])
    except Exception:
        obj = torch.load(path, map_location="cpu")
        return float(obj["t"])

class CE_Scorer:
    def __init__(self, model_path, temperature_path):
        self.model = CEModel(CFG.ce_backbone, 3, CFG.ce_dropout).to(CFG.device).eval()
        self.model.load_state_dict(torch.load(model_path, map_location=CFG.device))
        self.tok = tok_ce
        self.T = load_temperature(temperature_path)

    @torch.no_grad()
    def score(self, A_list, B_list):
        enc = self.tok(A_list, B_list, truncation=True, max_length=CFG.ce_max_len,
                       padding=True, return_tensors="pt")
        enc = {k:v.to(CFG.device) for k,v in enc.items()}
        logits = self.model(enc) / max(0.5, min(5.0, self.T))
        return logits.softmax(-1).detach().cpu().numpy()  # [:,0]=E, [:,1]=N, [:,2]=C

ce_scorer = CE_Scorer(os.path.join(CFG.outdir,"teacher.pt"),
                      os.path.join(CFG.outdir,"temp.json"))

def pack_dualbeam_pairaware(context: str, prompt: str, response: str) -> str:
    sents = sent_tokenize_vi(context)
    if not sents: return ""
    PR = join_pr(prompt, response)
    R, P = toks(response), toks(prompt)
    W = [toks(s) for s in sents]
    idf, avgdl = build_idf(W)

    spans=[]
    for k in CFG.span_sizes:
        for st in range(0, len(sents), CFG.stride_sent):
            en = min(len(sents), st+k)
            if en<=st: continue
            txt = " ".join(sents[st:en]); ET = toks(txt)
            slex = 0.5*(0.7*soft_char_overlap(R, ET) + 0.3*bm25_score(R, ET, idf, avgdl)) + \
                   0.5*(0.7*soft_char_overlap(P, ET) + 0.3*bm25_score(P, ET, idf, avgdl))
            spans.append((st,en,txt,slex))
    spans.sort(key=lambda x:x[3], reverse=True)
    pool = spans[:min(CFG.top_span_pool, len(spans))]
    if not pool:
        ids = tok_pack.encode(context, add_special_tokens=False)[:EFFECTIVE_MAX_SINGLE]
        return tok_pack.decode(ids, skip_special_tokens=True)

    A = [PR]*len(pool); B = [x[2] for x in pool]
    probs = ce_scorer.score(A,B)  # [:,0]=E, [:,1]=N, [:,2]=C

    r_nums = numbers(response); rsp_neg = has_neg(response)
    supp_list, conf_list = [], []
    for (st,en,txt,sx), (pe,pn,pc) in zip(pool, probs):
        t_nums = numbers(txt); txt_neg = has_neg(txt)
        num_match_all = (r_nums and r_nums.issubset(t_nums))
        num_mismatch  = (r_nums and t_nums and r_nums != t_nums)
        neg_flip      = (rsp_neg != txt_neg)

        supp = 0.75*pe - 0.25*pc + 0.10*sx
        if num_match_all: supp += CFG.boost_num
        if rsp_neg and txt_neg: supp += CFG.boost_neg

        conf = 0.75*pc - 0.10*pe + 0.05*sx
        if num_mismatch: conf += 0.25
        if neg_flip:     conf += 0.20

        supp_list.append((st,en,txt,sx,pe,pn,pc,supp))
        conf_list.append((st,en,txt,sx,pe,pn,pc,conf))

    supp_list.sort(key=lambda x: x[-1], reverse=True)
    conf_list.sort(key=lambda x: x[-1], reverse=True)

    used, mask = [], np.zeros(len(sents), dtype=int)
    cur, budget = 0, EFFECTIVE_MAX_SINGLE
    tgt_conf = CFG.conflict_target_ratio
    pi, pj = 0, 0
    conf_taken, total_taken = 0, 0

    def try_take(item):
        nonlocal cur
        st,en,txt,*_ = item[:4]
        tlen = len(tok_pack.encode(txt, add_special_tokens=False))
        if cur + tlen > budget: return False
        if mask[st:en].sum() > 0.5*(en-st): return False
        used.append((st,en,txt,tlen)); mask[st:en]=1; cur+=tlen
        return True

    while (pi < len(supp_list) or pj < len(conf_list)) and cur < 0.9*budget:
        ratio = (conf_taken/max(1,total_taken)) if total_taken>0 else 0.0
        pick_conf = (ratio < tgt_conf)
        took=False
        if pick_conf and pj < len(conf_list):
            if try_take(conf_list[pj]): conf_taken += 1; total_taken += 1; took=True
            pj += 1
        if not took and pi < len(supp_list):
            if try_take(supp_list[pi]): total_taken += 1; took=True
            pi += 1
        if not took and pj < len(conf_list):
            if try_take(conf_list[pj]): conf_taken += 1; total_taken += 1; took=True
            pj += 1
        if not took: break

    if conf_taken == 0 and len(conf_list) > 0:
        for item in conf_list:
            if try_take(item): break

    if not used:
        st,en,txt,_ = pool[0]
        ids = tok_pack.encode(txt, add_special_tokens=False)[:EFFECTIVE_MAX_SINGLE]
        return tok_pack.decode(ids, skip_special_tokens=True)

    used.sort(key=lambda x: x[0])
    evi = " ".join(u[2] for u in used)

    rl = rouge_l_lcs(toks(response), toks(evi))
    if rl < 0.18:
        best = max(pool, key=lambda x: rouge_l_lcs(toks(response), toks(x[2])))
        buf = tok_pack.encode(best[2], add_special_tokens=False)[:EFFECTIVE_MAX_SINGLE]
        evi2 = tok_pack.decode(buf, skip_special_tokens=True)
        if rouge_l_lcs(toks(response), toks(evi2)) > rl: evi = evi2
    return evi

In [10]:
def mass_from_evidence(context, prompt, response, evidence, topk=3):
    sents = sent_tokenize_vi(evidence)
    if not sents: return 0.0, 0.0, 0.0
    PR = join_pr(prompt, response)
    P = ce_scorer.score([PR]*len(sents), sents)   # [n,3] = E,N,C
    k = min(topk, len(sents))
    sup = float(np.mean(np.sort(P[:,0])[-k:]))
    con = float(np.mean(np.sort(P[:,2])[-k:]))
    neu = float(np.mean(np.sort(P[:,1])[-k:]))
    return sup, con, neu

def analyze_packer(df, pack_fn, name, save_dir=CFG.outdir, include_label=True, limit=None):
    rows=[]
    it = range(len(df)) if limit is None else range(min(limit, len(df)))
    for i in tqdm(it, desc=f"[{name}] pack & score"):
        r = df.iloc[i]
        ctx, prm, rsp = r[CFG.ctx_col], r[CFG.prm_col], r[CFG.rsp_col]
        evi = pack_fn(ctx, prm, rsp)
        rt, et = toks(rsp), toks(evi)
        uni = len(set(rt)&set(et))/max(1,len(set(rt)|set(et)))
        bi  = len(set(zip(rt,rt[1:])) & set(zip(et,et[1:]))) / \
              max(1, len(set(zip(rt,rt[1:])) | set(zip(et,et[1:]))))
        rl  = rouge_l_lcs(rt, et)
        if include_label:
            rows.append([str(r[CFG.id_col]), str(r[CFG.lab_col]), evi, uni, bi, rl])
        else:
            rows.append([str(r[CFG.id_col]), evi, uni, bi, rl])

    cols = [CFG.id_col, "label", "evidence", "uni_cov", "bi_cov", "rougeL"] if include_label \
           else [CFG.id_col, "evidence", "uni_cov", "bi_cov", "rougeL"]
    dfm = pd.DataFrame(rows, columns=cols)
    path = os.path.join(save_dir, f"{name}_coverage.csv")
    dfm.to_csv(path, index=False)
    print("Saved:", path)
    return dfm

def semantic_audit(df_raw, df_cov, split_name):
    raw_ix = df_raw.set_index(CFG.id_col)
    rows=[]
    for _, row in tqdm(df_cov.iterrows(), total=len(df_cov), desc=f"[semantic {split_name}]"):
        rid = str(row[CFG.id_col])
        base = raw_ix.loc[rid]
        sup, con, neu = mass_from_evidence(base[CFG.ctx_col], base[CFG.prm_col],
                                           base[CFG.rsp_col], row["evidence"], topk=3)
        rows.append([rid, row.get("label", None), sup, con, sup-con, neu, row["rougeL"], row.get("uni_cov", 0.0)])
    sem = pd.DataFrame(rows, columns=[CFG.id_col,"label","support_mass","conflict_mass","gap",
                                      "neutral_cov","rougeL_ctx","uni_cov_ctx"])
    out = os.path.join(CFG.outdir, f"{split_name}_semantic.csv"); sem.to_csv(out, index=False)
    print("Saved:", out)
    return sem

def gate_report(sem_df, name="split"):
    print(f"\n=== Means by class ({name}) ===")
    print(sem_df.groupby("label")[["support_mass","conflict_mass","neutral_cov","uni_cov_ctx","rougeL_ctx"]].mean())
    y_intr = (sem_df["label"]=="intrinsic").astype(int).to_numpy()
    y_ex   = (sem_df["label"]=="extrinsic").astype(int).to_numpy()
    auc_intr = roc_auc_score(y_intr, sem_df["conflict_mass"].to_numpy())
    auc_ex   = roc_auc_score(y_ex, (sem_df["neutral_cov"] - sem_df["support_mass"]).to_numpy())
    rl_med   = float(sem_df["rougeL_ctx"].median())
    print(f"AUC(intrinsic via conflict_mass) ≈ {auc_intr:.3f}")
    print(f"AUC(extrinsic via neutral-support) ≈ {auc_ex:.3f}")
    print(f"ROUGE-L median ≈ {rl_med:.3f}")
    return auc_intr, auc_ex, rl_med

def meta_decider_eval(sem_df, name="split"):
    X = sem_df[["support_mass","conflict_mass","neutral_cov","rougeL_ctx","uni_cov_ctx"]].to_numpy()
    y = sem_df["label"].map(LABEL2ID).to_numpy()
    clf = LogisticRegression(max_iter=200).fit(X,y)
    pred = clf.predict(X)
    f1 = f1_score(y, pred, average="macro"); acc = accuracy_score(y,pred)
    print(f"[Meta-decider {name}] Macro-F1={f1:.4f} Acc={acc:.4f}")
    return clf, f1, acc

# Split train/val và chạy
sss = StratifiedShuffleSplit(n_splits=1, test_size=CFG.valid_ratio, random_state=CFG.seed)
tr_idx, va_idx = next(sss.split(train_df, train_df[CFG.lab_col]))
tr_df = train_df.iloc[tr_idx].reset_index(drop=True)
va_df = train_df.iloc[va_idx].reset_index(drop=True)

cov_tr = analyze_packer(tr_df, pack_dualbeam_pairaware, "hybrid_train_v3", include_label=True)
cov_va = analyze_packer(va_df, pack_dualbeam_pairaware, "hybrid_val_v3",   include_label=True)

sem_tr = semantic_audit(tr_df, cov_tr, "train_v3")
sem_va = semantic_audit(va_df, cov_va, "val_v3")

auc_tr_i, auc_tr_e, rl_tr = gate_report(sem_tr, "train_v3")
auc_va_i, auc_va_e, rl_va = gate_report(sem_va, "val_v3")

meta_tr, f1_tr, _ = meta_decider_eval(sem_tr, "train_v3")
meta_va, f1_va, _ = meta_decider_eval(sem_va, "val_v3")

gate_ok = bool(
    float(auc_va_i) >= float(CFG.auc_intrinsic_min) and
    float(auc_va_e) >= float(CFG.auc_extrinsic_min) and
    float(f1_va)    >= float(CFG.meta_f1_min)       and
    float(rl_va)    >= float(CFG.rl_median_min)
)

json.dump({
    "auc_intr_train": float(auc_tr_i),
    "auc_extr_train": float(auc_tr_e),
    "rl_med_train":   float(rl_tr),
    "auc_intr_val":   float(auc_va_i),
    "auc_extr_val":   float(auc_va_e),
    "rl_med_val":     float(rl_va),
    "meta_f1_train":  float(f1_tr),
    "meta_f1_val":    float(f1_va),
    "gate_pass":      bool(gate_ok),
}, open(os.path.join(CFG.outdir, "gate_report.json"), "w"), ensure_ascii=False, indent=2)

print("\n[GATE] PASS? ", gate_ok,
      f"(AUC_i {auc_va_i:.3f} / {CFG.auc_intrinsic_min}, AUC_e {auc_va_e:.3f} / {CFG.auc_extrinsic_min}, "
      f"F1_meta {f1_va:.3f} / {CFG.meta_f1_min}, RL_med {rl_va:.3f} / {CFG.rl_median_min})")

[hybrid_train_v3] pack & score:   0%|          | 0/6300 [00:00<?, ?it/s]

Saved: /kaggle/working/ce_gate_pipeline_v3/hybrid_train_v3_coverage.csv


[hybrid_val_v3] pack & score:   0%|          | 0/700 [00:00<?, ?it/s]

Saved: /kaggle/working/ce_gate_pipeline_v3/hybrid_val_v3_coverage.csv


[semantic train_v3]:   0%|          | 0/6300 [00:00<?, ?it/s]

Saved: /kaggle/working/ce_gate_pipeline_v3/train_v3_semantic.csv


[semantic val_v3]:   0%|          | 0/700 [00:00<?, ?it/s]

Saved: /kaggle/working/ce_gate_pipeline_v3/val_v3_semantic.csv

=== Means by class (train_v3) ===
           support_mass  conflict_mass  neutral_cov  uni_cov_ctx  rougeL_ctx
label                                                                       
extrinsic      0.258124       0.321481     0.509798     0.248837    0.235355
intrinsic      0.037195       0.359927     0.660521     0.274342    0.254092
no             0.984094       0.006603     0.016470     0.332091    0.317218
AUC(intrinsic via conflict_mass) ≈ 0.667
AUC(extrinsic via neutral-support) ≈ 0.625
ROUGE-L median ≈ 0.242

=== Means by class (val_v3) ===
           support_mass  conflict_mass  neutral_cov  uni_cov_ctx  rougeL_ctx
label                                                                       
extrinsic      0.294229       0.293933     0.521417     0.248871    0.238094
intrinsic      0.033143       0.373569     0.647033     0.268434    0.245356
no             0.980133       0.009732     0.016447     0.331816    0

In [11]:
def attach_mass(df_raw, df_cov, split_name):
    raw_ix = df_raw.set_index(CFG.id_col)
    rows=[]
    for _, row in tqdm(df_cov.iterrows(), total=len(df_cov), desc=f"[attach mass {split_name}]"):
        rid = str(row[CFG.id_col])
        base = raw_ix.loc[rid]
        sup, con, neu = mass_from_evidence(base[CFG.ctx_col], base[CFG.prm_col],
                                           base[CFG.rsp_col], row["evidence"], topk=3)
        rows.append([rid, sup, con, neu])
    massdf = pd.DataFrame(rows, columns=[CFG.id_col,"support_mass","conflict_mass","neutral_cov"])
    merged = df_cov.merge(massdf, on=CFG.id_col, how="left")
    path = os.path.join(CFG.outdir, f"{split_name}_coverage_with_mass.csv")
    merged.to_csv(path, index=False)
    print("Saved:", path)
    return merged

cov_tr_m = attach_mass(tr_df, cov_tr, "hybrid_train_v3")
cov_va_m = attach_mass(va_df, cov_va, "hybrid_val_v3")

# TEST coverage + mass (không có label, có cột id gốc)
cov_te = analyze_packer(test_df, pack_dualbeam_pairaware, "hybrid_test_v3", include_label=False)
raw_ix_te = test_df.set_index(CFG.id_col)
rows=[]
for _, row in tqdm(cov_te.iterrows(), total=len(cov_te), desc="[attach mass TEST]"):
    rid = str(row[CFG.id_col])
    base = raw_ix_te.loc[rid]
    sup, con, neu = mass_from_evidence(base[CFG.ctx_col], base[CFG.prm_col],
                                       base[CFG.rsp_col], row["evidence"], topk=3)
    rows.append([rid, sup, con, neu])
mass_te = pd.DataFrame(rows, columns=[CFG.id_col,"support_mass","conflict_mass","neutral_cov"])
cov_te_m = cov_te.merge(mass_te, on=CFG.id_col, how="left")
out_test = os.path.join(CFG.outdir, "hybrid_test_v3_coverage_with_mass.csv")
cov_te_m.to_csv(out_test, index=False)
print("Saved TEST coverage+mass:", out_test)

[attach mass hybrid_train_v3]:   0%|          | 0/6300 [00:00<?, ?it/s]

Saved: /kaggle/working/ce_gate_pipeline_v3/hybrid_train_v3_coverage_with_mass.csv


[attach mass hybrid_val_v3]:   0%|          | 0/700 [00:00<?, ?it/s]

Saved: /kaggle/working/ce_gate_pipeline_v3/hybrid_val_v3_coverage_with_mass.csv


[hybrid_test_v3] pack & score:   0%|          | 0/2000 [00:00<?, ?it/s]

Saved: /kaggle/working/ce_gate_pipeline_v3/hybrid_test_v3_coverage.csv


[attach mass TEST]:   0%|          | 0/2000 [00:00<?, ?it/s]

Saved TEST coverage+mass: /kaggle/working/ce_gate_pipeline_v3/hybrid_test_v3_coverage_with_mass.csv


In [12]:
# label map
json.dump({"LABEL2ID": LABEL2ID, "ID2LABEL": ID2LABEL},
          open(os.path.join(CFG.outdir, "labels.json"), "w"),
          ensure_ascii=False, indent=2)

artifacts = {
    "teacher_ckpt": "teacher.pt",
    "teacher_temp": "temp.json",
    "pairs_raw": "ce_pairs_raw.csv",
    "pairs_balanced": "ce_pairs_balanced.csv",
    "coverage_train": "hybrid_train_v3_coverage.csv",
    "coverage_val": "hybrid_val_v3_coverage.csv",
    "semantic_train": "train_v3_semantic.csv",
    "semantic_val": "val_v3_semantic.csv",
    "coverage_train_with_mass": "hybrid_train_v3_coverage_with_mass.csv",
    "coverage_val_with_mass": "hybrid_val_v3_coverage_with_mass.csv",
    "coverage_test_with_mass": "hybrid_test_v3_coverage_with_mass.csv",
    "gate_report": "gate_report.json",
    "label_map": "labels.json",
}

json.dump({"artifacts": artifacts,
           "notes": "All coverage/mass files are keyed by the RAW id (no idx)."},
          open(os.path.join(CFG.outdir, "MANIFEST.json"), "w"),
          ensure_ascii=False, indent=2)

readme = f"""
Artifacts to reuse in the classifier notebook:

- CE teacher: teacher.pt (+ temp.json)
- Packer coverage (id-based): hybrid_train_v3_coverage.csv, hybrid_val_v3_coverage.csv
- Semantic: train_v3_semantic.csv, val_v3_semantic.csv
- Coverage+mass (id-based): hybrid_*_coverage_with_mass.csv (train/val/test)
- Gate report: gate_report.json
- Label maps: labels.json

All files are joined by RAW '{CFG.id_col}' — no 'idx' used anywhere.
"""
open(os.path.join(CFG.outdir, "README.txt"), "w", encoding="utf-8").write(readme)

print("Saved MANIFEST & README to:", CFG.outdir)
print(json.dumps(artifacts, indent=2))

Saved MANIFEST & README to: /kaggle/working/ce_gate_pipeline_v3
{
  "teacher_ckpt": "teacher.pt",
  "teacher_temp": "temp.json",
  "pairs_raw": "ce_pairs_raw.csv",
  "pairs_balanced": "ce_pairs_balanced.csv",
  "coverage_train": "hybrid_train_v3_coverage.csv",
  "coverage_val": "hybrid_val_v3_coverage.csv",
  "semantic_train": "train_v3_semantic.csv",
  "semantic_val": "val_v3_semantic.csv",
  "coverage_train_with_mass": "hybrid_train_v3_coverage_with_mass.csv",
  "coverage_val_with_mass": "hybrid_val_v3_coverage_with_mass.csv",
  "coverage_test_with_mass": "hybrid_test_v3_coverage_with_mass.csv",
  "gate_report": "gate_report.json",
  "label_map": "labels.json"
}


In [16]:
# Cần thêm dataset chứa file /kaggle/input/kaggle-json/kaggle.json
import subprocess, shutil, stat

DATA_DIR = CFG.outdir
META = {
  "title": "CE Gate Pipeline v32 (teacher+coverage+mass)",
  "id": "honghien123/ce-gate-pipeline-v3-3",
  "licenses": [{"name": "CC0-1.0"}],
  "isPrivate": True
}
open(os.path.join(DATA_DIR, "dataset-metadata.json"), "w").write(json.dumps(META, ensure_ascii=False, indent=2))

def run(cmd):
    r = subprocess.run(cmd, capture_output=True, text=True)
    print(r.stdout or r.stderr)
    return r.returncode

try:
    os.makedirs("/root/.kaggle", exist_ok=True)
    shutil.copy("/kaggle/input/kaggle-json-2/kaggle.json", "/root/.kaggle/kaggle.json")
    os.chmod("/root/.kaggle/kaggle.json", stat.S_IRUSR | stat.S_IWUSR)
    rc = run(["kaggle", "datasets", "create", "-p", DATA_DIR, "-r", "zip"])
    if rc != 0:
        run(["kaggle", "datasets", "version", "-p", DATA_DIR, "-m", "update"])
except Exception as e:
    print("Skip Kaggle CLI publish:", e)

Starting upload for file gate_report.json
Error while trying to load upload info: ApiStartBlobUploadRequest.__init__() got an unexpected keyword argument 'type'
Upload successful: gate_report.json (331B)
Starting upload for file hybrid_train_v3_coverage_with_mass.csv
Error while trying to load upload info: ApiStartBlobUploadRequest.__init__() got an unexpected keyword argument 'type'
Upload successful: hybrid_train_v3_coverage_with_mass.csv (4MB)
Starting upload for file MANIFEST.json
Error while trying to load upload info: ApiStartBlobUploadRequest.__init__() got an unexpected keyword argument 'type'
Upload successful: MANIFEST.json (727B)
Starting upload for file labels.json
Error while trying to load upload info: ApiStartBlobUploadRequest.__init__() got an unexpected keyword argument 'type'
Upload successful: labels.json (154B)
Starting upload for file temp.json
Error while trying to load upload info: ApiStartBlobUploadRequest.__init__() got an unexpected keyword argument 'type'
Upl